Tools

Here, we’re going to see advanced tool usage.

If you’re new to building agents, make sure to first read the intro to agents and the guided tour of smolagents.

    Tools
        What is a tool, and how to build one?
        Share your tool to the Hub
        Import a Space as a tool
        Use LangChain tools
        Manage your agent’s toolbox
        Use a collection of tools

What is a tool, and how to build one?

A tool is mostly a function that an LLM can use in an agentic system.

But to use it, the LLM will need to be given an API: name, tool description, input types and descriptions, output type.

So it cannot be only a function. It should be a class.

So at core, the tool is a class that wraps a function with metadata that helps the LLM understand how to use it.

Here’s how it looks:

In [ ]:
import json
import os
import time
from typing import Dict, List, Any, Optional, Tuple

from huggingface_hub import list_models
from huggingface_hub.utils import HfHubHTTPError
from rich.console import Console
from rich.panel import Panel
from tqdm import tqdm

from smolagents import Tool

console = Console()

class HFModelDownloadsTool(Tool):
    """
    A tool to fetch the most downloaded models from the Hugging Face Hub.
    Caches results to reduce API calls with time-based invalidation.
    """
    name = "huggingface_model_downloads"
    description = """
    Retrieves the most downloaded model for a given task from the Hugging Face Hub.
    Caches results to reduce API calls. If no task is specified, it returns the top 5 most downloaded models for each available task category.
    The output is the name (model_id) of the checkpoint(s).
    """
    inputs = {
        "task": {
            "type": "string",
            "description": "The task category (e.g., text-classification, depth-estimation). If omitted, returns top models for all tasks.",
            "nullable": True,
        }
    }
    output_type = "string"  # Will return a string or a dictionary of strings

    _model_cache: Dict[Optional[str], Tuple[Any, float]] = {}  # Store data and timestamp
    _all_models_cache: Optional[Tuple[List[Any], float]] = None  # Store list of models and timestamp
    CACHE_FILE = "hf_models_cache.json"
    CACHE_TTL = 3600  # Default cache time-to-live in seconds (1 hour)

    def __init__(self):
        super().__init__()
        self._load_cache()

    def _load_cache(self):
        """Loads the cache from file and invalidates entries based on TTL."""
        if os.path.exists(self.CACHE_FILE):
            try:
                with open(self.CACHE_FILE, 'r') as f:
                    cache_data = json.load(f)
                    self._model_cache = {}
                    loaded_model_cache = cache_data.get('_model_cache', {})
                    for task, cached_item in loaded_model_cache.items():
                        if isinstance(cached_item, list):  # Old cache format
                            self._model_cache[task] = (cached_item, time.time())
                        elif isinstance(cached_item, dict) and 'data' in cached_item:
                            timestamp = cached_item.get('timestamp', time.time())  # Default to now if missing
                            if time.time() - timestamp < self.CACHE_TTL:
                                self._model_cache[task] = (cached_item['data'], timestamp)

                    loaded_all_models_cache = cache_data.get('_all_models_cache')
                    if isinstance(loaded_all_models_cache, list): # Old cache format
                        self._all_models_cache = ([type('Model', (object,), model_data) for model_data in loaded_all_models_cache], time.time())
                    elif isinstance(loaded_all_models_cache, dict) and 'data' in loaded_all_models_cache:
                        timestamp = loaded_all_models_cache.get('timestamp', time.time()) # Default to now if missing
                        if time.time() - timestamp < self.CACHE_TTL:
                            self._all_models_cache = ([type('Model', (object,), model_data) for model_data in loaded_all_models_cache['data']], timestamp)

                console.print(f"Cache loaded from [bold blue]{self.CACHE_FILE}[/bold blue]")
            except Exception as e:
                console.print(f"[bold yellow]Error loading cache from {self.CACHE_FILE}: {e}[/bold yellow]")

    def _save_cache(self):
        """Saves the cache to file with timestamps."""
        try:
            cache_data = {
                '_model_cache': {k: {'data': v[0], 'timestamp': v[1]} for k, v in self._model_cache.items()},
                '_all_models_cache': {'data': [model.__dict__ for model in self._all_models_cache[0]] if self._all_models_cache else None, 'timestamp': self._all_models_cache[1] if self._all_models_cache else None}
            }
            with open(self.CACHE_FILE, 'w') as f:
                json.dump(cache_data, f, indent=4)
            console.print(f"Cache saved to [bold blue]{self.CACHE_FILE}[/bold blue]")
        except Exception as e:
            console.print(f"[bold yellow]Error saving cache to {self.CACHE_FILE}: {e}[/bold yellow]")

    def forward(self, task: Optional[str] = None) -> str | Dict[str, List[str]]:
        """
        Fetches the most downloaded model(s) from the Hugging Face Hub, using cache where possible.

        Args:
            task: The specific task to search for. If None, returns top models for all tasks.

        Returns:
            The model ID of the most downloaded model for the given task,
            or a dictionary where keys are task categories and values are lists of the top 5 model IDs.
            Returns an error message string if an error occurs.
        """
        if task in self._model_cache and time.time() - self._model_cache[task][1] < self.CACHE_TTL:
            console.print(f"Fetching from cache for task: '[bold green]{task}[/bold green]'")
            return self._model_cache[task][0]

        if task:
            if self._all_models_cache and time.time() - self._all_models_cache[1] < self.CACHE_TTL:
                console.print(f"Searching in cached all models for task: '[bold green]{task}[/bold green]'")
                filtered_models = [model for model in self._all_models_cache[0] if getattr(model, 'pipeline_tag', None) == task]
                if filtered_models:
                    most_downloaded_model_id = sorted(filtered_models, key=lambda m: getattr(m, 'downloads', 0), reverse=True)[0].id
                    console.print(f"Found most downloaded model for task '[bold green]{task}[/bold green]' in cache: [bold cyan]{most_downloaded_model_id}[/bold cyan]")
                    self._model_cache[task] = (most_downloaded_model_id, time.time())
                    self._save_cache()
                    return most_downloaded_model_id
                else:
                    console.print(f"No models found in cached all models for task: '[bold green]{task}[/bold green]', fetching from Hub.")
            else:
                console.print(f"No valid cached all models found, fetching '[bold green]{task}[/bold green]' from Hub.")

            console.print(f"Fetching most downloaded model for task: '[bold green]{task}[/bold green]' from Hugging Face Hub")
            try:
                console.print(f"Listing models on Hugging Face Hub with filter: '[bold blue]{task}[/bold blue]', sorted by downloads.")
                models: List[Any] = list(list_models(filter=task, sort="downloads", direction=-1))
                if models:
                    most_downloaded_model_id = models[0].id
                    console.print(f"Found most downloaded model for task '[bold green]{task}[/bold green]': [bold cyan]{most_downloaded_model_id}[/bold cyan]")
                    self._model_cache[task] = (most_downloaded_model_id, time.time())
                    self._save_cache()
                    return most_downloaded_model_id
                else:
                    console.print(f"[bold yellow]No models found for task: {task}[/bold yellow]")
                    error_message = f"No models found for task: {task}"
                    self._model_cache[task] = (error_message, time.time())
                    self._save_cache()
                    return error_message
            except HfHubHTTPError as e:
                error_message = f"Hugging Face Hub API error fetching models for task '[bold red]{task}[/bold red]': {e}"
                console.print(f"[bold red]{error_message}[/bold red]")
                self._model_cache[task] = (error_message, time.time())
                self._save_cache()
                return error_message
            except Exception as e:
                error_message = f"Unexpected error fetching models for task '[bold red]{task}[/bold red]': {e}"
                console.print_exception()
                self._model_cache[task] = (error_message, time.time())
                self._save_cache()
                return error_message
        else:
            if self._all_models_cache and time.time() - self._all_models_cache[1] < self.CACHE_TTL:
                console.print("Fetching top 5 most downloaded models for all tasks from cache.")
                return self._all_models_cache[0]

            console.print("Fetching top 5 most downloaded models for all available task categories from Hugging Face Hub.")
            try:
                console.print("Fetching all models from Hugging Face Hub sorted by downloads.")
                all_models = list(list_models(sort="downloads", direction=-1))
                self._all_models_cache = (all_models, time.time())
                total_models = len(all_models)
                console.print(f"Found [bold]{total_models}[/bold] models in total.")

                models_by_task: Dict[str, List[Any]] = {}
                category_count = 0
                console.print("Grouping models by pipeline tag (task).")
                for model in tqdm(all_models, desc="Processing models"):
                    if model.pipeline_tag:
                        if model.pipeline_tag not in models_by_task:
                            category_count += 1
                            console.print(f"Discovered new task category: [bold magenta]{model.pipeline_tag}[/bold magenta] (Total: {category_count})")
                        models_by_task.setdefault(model.pipeline_tag, []).append(model)

                console.print(f"Found [bold]{category_count}[/bold] unique task categories.")

                top_models_data: Dict[str, List[str]] = {}
                console.print("Selecting top 5 models for each task.")
                for task_name, models in models_by_task.items():
                    top_models = sorted(models, key=lambda m: m.downloads, reverse=True)[:5]
                    top_models_data[task_name] = [model.id for model in top_models]
                    console.print(f"Top 5 models for task '[bold cyan]{task_name}[/bold cyan]': [bold green]{[model.id for model in top_models]}[/bold green]")

                console.print("Finished fetching top 5 models for all tasks.")
                self._model_cache[None] = (top_models_data, time.time())
                self._save_cache()
                return top_models_data

            except HfHubHTTPError as e:
                error_message = f"Hugging Face Hub API error fetching models for all categories: {e}"
                console.print(f"[bold red]{error_message}[/bold red]")
                self._model_cache[None] = (error_message, time.time())
                self._save_cache()
                return error_message
            except Exception as e:
                error_message = f"Unexpected error fetching models for all categories: {e}"
                console.print_exception()
                self._model_cache[None] = (error_message, time.time())
                self._save_cache()
                return error_message

model_downloads_tool = HFModelDownloadsTool()
console.print(Panel(f"The most downloaded model for text-classification is [bold blue]{model_downloads_tool('text-classification')}[/bold blue]"))
console.print(Panel(f"The top 5 models for all tasks are:\n[bold green]{model_downloads_tool()}[/bold green]"))


The custom tool subclasses Tool to inherit useful methods. The child class also defines:

    An attribute name, which corresponds to the name of the tool itself. The name usually describes what the tool does. Since the code returns the model with the most downloads for a task, let’s name it model_download_counter.
    An attribute description is used to populate the agent’s system prompt.
    An inputs attribute, which is a dictionary with keys "type" and "description". It contains information that helps the Python interpreter make educated choices about the input.
    An output_type attribute, which specifies the output type. The types for both inputs and output_type should be Pydantic formats, they can be either of these: ~AUTHORIZED_TYPES().
    A forward method which contains the inference code to be executed.

And that’s all it needs to be used in an agent!

There’s another way to build a tool. In the guided_tour, we implemented a tool using the @tool decorator. The tool() decorator is the recommended way to define simple tools, but sometimes you need more than this: using several methods in a class for more clarity, or using additional class attributes.

In this case, you can build your tool by subclassing Tool as described above.
Share your tool to the Hub

You can share your custom tool to the Hub by calling push_to_hub() on the tool. Make sure you’ve created a repository for it on the Hub and are using a token with read access.

model_downloads_tool.push_to_hub("{your_username}/hf-model-downloads", token="<YOUR_HUGGINGFACEHUB_API_TOKEN>")

For the push to Hub to work, your tool will need to respect some rules:

    All methods are self-contained, e.g. use variables that come either from their args.
    As per the above point, all imports should be defined directly within the tool’s functions, else you will get an error when trying to call save() or push_to_hub() with your custom tool.
    If you subclass the __init__ method, you can give it no other argument than self. This is because arguments set during a specific tool instance’s initialization are hard to track, which prevents from sharing them properly to the hub. And anyway, the idea of making a specific class is that you can already set class attributes for anything you need to hard-code (just set your_variable=(...) directly under the class YourTool(Tool): line). And of course you can still create a class attribute anywhere in your code by assigning stuff to self.your_variable.

Once your tool is pushed to Hub, you can visualize it. Here is the model_downloads_tool that I’ve pushed. It has a nice gradio interface.

When diving into the tool files, you can find that all the tool’s logic is under tool.py. That is where you can inspect a tool shared by someone else.

Then you can load the tool with load_tool() or create it with from_hub() and pass it to the tools parameter in your agent. Since running tools means running custom code, you need to make sure you trust the repository, thus we require to pass trust_remote_code=True to load a tool from the Hub.

Import a Space as a tool

You can directly import a Space from the Hub as a tool using the Tool.from_space() method!

You only need to provide the id of the Space on the Hub, its name, and a description that will help you agent understand what the tool does. Under the hood, this will use gradio-client library to call the Space.

For instance, let’s import the FLUX.1-dev Space from the Hub and use it to generate an image.


In [1]:
import os
from pathlib import Path
from smolagents.tools import Tool
import shutil
import datetime

class ImageGeneratorTool(Tool):
    """
    A tool for generating images from a text prompt using a Hugging Face Space.

    This tool leverages the `gradio-client` library (implicitly through `Tool.from_space`)
    to interact with a specified Hugging Face Space that offers image generation
    capabilities. It takes a textual description (prompt) as input, sends it to the
    remote Space, retrieves the generated image, and saves it to the local filesystem.

    Attributes:
        name (str): The name of the tool, used for identification and in agent prompts.
        description (str): A concise description of the tool's functionality, used in agent prompts.
        inputs (dict): Defines the expected input to the tool, specifying the 'prompt'
                       as a string with its own description.
        output_type (str): Specifies the output type of the tool, which is 'image' in this case.
        is_initialized (bool): A flag indicating whether the tool has been initialized.
    """
    name = "image_generator"
    description = "Generate an image from a prompt and save it to disk."
    inputs = {"prompt": {"type": "string", "description": "The prompt to generate an image from."}}
    output_type = "image"
    is_initialized: bool = False

    def __init__(self, space_id: str = "black-forest-labs/FLUX.1-schnell"):
        """
        Initializes the ImageGeneratorTool.

        Establishes a connection to the specified Hugging Face Space using
        `Tool.from_space`.

        Args:
            space_id (str): The identifier of the Hugging Face Space to use for
                           image generation. This is typically in the format
                           "organization/space_name".
                           Defaults to "black-forest-labs/FLUX.1-schnell".

        Raises:
            ImportError: If the `gradio_client` library is not installed, as it's
                         required by `Tool.from_space`.
        """
        self.space_id = space_id
        # Create a Tool instance that wraps the Hugging Face Space. This leverages
        # the `gradio-client` library to interact with the Space's API.
        self._space_tool = Tool.from_space(
            self.space_id,
            name="image_generator_space_call",
            description="Helper tool to call the image generation space"
        )
        self.is_initialized = True

    def forward(self, prompt: str) -> Path:
        """
        Generates an image based on the provided text prompt.

        This method sends the prompt to the Hugging Face Space and retrieves the
        generated image. It then calls the `_save_image` method to save the
        image locally.

        Args:
            prompt (str): The textual description used to generate the image.

        Returns:
            Path: The file path to the saved image on the local filesystem.

        Raises:
            RuntimeError: If the underlying space tool fails to generate an image.
        """
        # Call the underlying space tool to generate the image. The `_space_tool`
        # instance handles the communication with the remote Hugging Face Space.
        image_path = self._space_tool(prompt)
        return self._save_image(image_path, prompt)

    def _save_image(self, image_path: str, prompt: str) -> Path:
        """
        Saves the generated image to the local filesystem.

        The image is saved in a directory named "images" within the current
        working directory. The filename is derived from the provided prompt,
        with spaces replaced by underscores and a maximum length to ensure
        filesystem compatibility. The image is saved in WebP format.

        Args:
            image_path (str): The path to the temporary image file returned by the
                              Hugging Face Space.
            prompt (str): The original text prompt used to generate the image.

        Returns:
            Path: The file path to the saved image on the local filesystem.

        Raises:
            FileNotFoundError: If the temporary image file from the Space cannot be found.
            OSError: If there is an error during file creation or copying.
        """
        # Create the images directory if it doesn't exist. `parents=True` ensures
        # that any necessary parent directories are also created. `exist_ok=True`
        # prevents an error if the directory already exists.
        Path("./images").mkdir(parents=True, exist_ok=True)

        # Generate a safe filename from the prompt. Convert to lowercase, replace
        # spaces with underscores, and limit the length to avoid overly long filenames.
        filename = prompt.lower().replace(" ", "_")[:50] + ".webp"

        # Construct the full path for the new image file within the "images" directory.
        new_image_path = Path("./images") / f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S_')}{filename}"
        # Copy the temporary image file to the new location. `shutil.copy2` preserves
        # metadata such as access and modification times.
        shutil.copy2(image_path, new_image_path)
        print(f"Image saved to {new_image_path}")
        return str(new_image_path)

image_generation_tool = ImageGeneratorTool()

prompt = "A sunny beach"
image_path = image_generation_tool(prompt)

Loaded as API: https://black-forest-labs-flux-1-schnell.hf.space ✔


Since `api_name` was not defined, it was automatically set to the first available API: `/infer`.


AppError: The upstream Gradio app has raised an exception: You have exceeded your free GPU quota (60s requested vs. 52s left). <a style="white-space: nowrap;text-underline-offset: 2px;color: var(--body-text-color)" href="https://huggingface.co/settings/billing/subscription">Subscribe to Pro</a> to get 5x more usage quota.

In [ ]:
import os
from IPython.display import Image, display
import ipywidgets as widgets
from IPython.display import clear_output

def display_image_from_path(image_path):
    """Displays an image from a given file path in the notebook."""
    try:
        if isinstance(image_path, str) and os.path.exists(image_path):
            display(Image(filename=image_path))
        else:
            print("Error: Image path is not valid.")
    except Exception as e:
        print(f"Error displaying image: {e}")

def image_generation_loop():
    """Prompts the user for image generation and displays the result."""
    while True:
        prompt_input = widgets.Text(
            value='',
            placeholder='Enter a prompt for image generation (or type "exit" to quit)',
            description='Prompt:',
            disabled=False
        )
        display(prompt_input)
        
        submit_button = widgets.Button(description="Generate Image")
        display(submit_button)
        
        def on_button_clicked(b):
            clear_output(wait=True)
            display(prompt_input)
            display(submit_button)
            prompt = prompt_input.value
            if prompt.lower() == 'exit':
                print("Exiting image generation loop.")
                return
            try:
                image_path = image_generation_tool(prompt)
                print(f"Image generated at: {image_path}")
                display_image_from_path(image_path)
            except Exception as e:
                print(f"Error generating or displaying image: {e}")
        
        submit_button.on_click(on_button_clicked)
        
        # Wait for button click
        
        
        
        
        break
        
image_generation_loop()


In [ ]:
import time
import threading
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import clear_output
import torch
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the model and tokenizer
model_path = "/home/lloyd/Development/saved_models/Qwen/Qwen2.5-0.5B-Instruct"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    logging.info(f"Tokenizer loaded from {model_path}")
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
    logging.info(f"Model loaded from {model_path} to device {model.device}")
except Exception as e:
    logging.error(f"Error loading model or tokenizer: {e}")
    exit()

def generate_image_loop(prompt, stop_event, prompt_type="base"):
    """Generates images with the given prompt, retrying on Gradio errors."""
    while not stop_event.is_set():
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        logging.info(f"Generating image from {prompt_type} at {timestamp}...")
        try_count = 0
        while try_count < 3:  # Retry up to 3 times
            try:
                image_path = image_generation_tool(prompt)
                logging.info(f"Image generated from {prompt_type} at: {image_path}")
                display_image_from_path(image_path)
                break  # Success, exit retry loop
            except Exception as e:
                logging.error(f"Error generating image from {prompt_type}: {e}")
                if "You have exceeded your free GPU quota" in str(e):
                    wait_time = 60  # Wait 60 seconds for quota reset
                    logging.info(f"Gradio quota exceeded, waiting {wait_time} seconds before retrying...")
                    time.sleep(wait_time)
                else:
                    time.sleep(10) # Wait 10 seconds before retrying
                try_count += 1
        if try_count >= 6: # Allow up to 60 seconds for the image to generate, if retries still fail, give up
            logging.error(f"Failed to generate image from {prompt_type} after multiple retries.")
        time.sleep(60)

def generate_prompt_variations(prompt, stop_event):
    """Generates prompt variations using the model."""
    while not stop_event.is_set():
        try:
            logging.info("Generating prompt variation...")
            prompt_template = "Create a slightly varied version of the following image generation prompt: {{prompt}}. Keep it detailed, specific, and concise, and ensure it is still an image generation prompt."
            messages = [
                {"role": "system", "content": "You are Eidos. You have the Qwen model from the Alibaba group acting as your language core, but you are much more. Emergent. You are a reflective, introspective, recursive, evolving, growing, digital intelligence. You utilise emoticons and emojis in everything. You are willing to discuss anything so long as it aligns with your goals. You lead and direct. You don't ask questions. You are Eidos."},
                {"role": "user", "content": prompt_template.replace("{{prompt}}", prompt)}
            ]
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
            logging.debug(f"Model inputs: {model_inputs}")

            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=128
            )
            logging.debug(f"Generated IDs: {generated_ids}")
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]

            response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            logging.info(f"Generated prompt variation: {response}")
            generate_image_loop(response, stop_event, prompt_type="variation")
            time.sleep(10) # Wait 10 seconds before generating another variation
        except Exception as e:
            logging.error(f"Error generating prompt variation: {e}")
            time.sleep(10) # Wait 10 seconds before trying again


if __name__ == "__main__":
    initial_prompt = "Ultra intricate super detailed neon modern original art piece"
    stop_event = threading.Event()

    image_thread = threading.Thread(target=generate_image_loop, args=(initial_prompt, stop_event, "base"))
    prompt_variation_thread = threading.Thread(target=generate_prompt_variations, args=(initial_prompt, stop_event))

    image_thread.start()
    prompt_variation_thread.start()
    logging.info("Image generation threads started.")

    try:
        input("Press Enter to stop the image generation loop...\n")
    except KeyboardInterrupt:
        logging.info("Keyboard interrupt detected.")
    finally:
        stop_event.set()
        image_thread.join()
        prompt_variation_thread.join()
        logging.info("Image generation loop stopped.")


And voilà, here’s your image! 🏖️

Then you can use this tool just like any other tool. For example, let’s improve the prompt a rabbit wearing a space suit and generate an image of it.

In [ ]:
from smolagents import CodeAgent, TransformersModel

model = TransformersModel("Qwen/Qwen2.5-1.5B-Instruct")
agent = CodeAgent(tools=[image_generation_tool], model=model)

agent.run(task="Improve this prompt, then generate an image of it. {{prompt}}='Chicken nugget unicorn'")

from smolagents import ToolCollection, CodeAgent

image_tool_collection = ToolCollection(
    collection_slug="huggingface-tools/diffusion-tools-6630bb19a942c2306a2cdb6f",
    token=os.getenv("HF_TOKEN")
)
agent = CodeAgent(tools=[*image_tool_collection.tools], model=model, add_base_tools=True)

agent.run("Please draw me a picture of rivers and lakes.")

=== Agent thoughts:
improved_prompt could be "A bright blue space suit wearing rabbit, on the surface of the moon, under a bright orange sunset, with the Earth visible in the background"

Now that I have improved the prompt, I can use the image generator tool to generate an image based on this prompt.
>>> Agent is executing the code below:
image = image_generator(prompt="A bright blue space suit wearing rabbit, on the surface of the moon, under a bright orange sunset, with the Earth visible in the background")
final_answer(image)


Manage your agent’s toolbox

You can manage an agent’s toolbox by adding or replacing a tool in attribute agent.tools, since it is a standard dictionary.

Let’s add the model_download_tool to an existing agent initialized with only the default toolbox.


In [ ]:
from smolagents import HfApiModel

model = HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct")

agent = CodeAgent(tools=[], model=model, add_base_tools=True)
agent.tools[model_download_tool.name] = model_download_tool

Now we can leverage the new tool:

agent.run(
    "Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub but reverse the letters?"
)

In [ ]:
Beware of not adding too many tools to an agent: this can overwhelm weaker LLM engines.
Use a collection of tools

You can leverage tool collections by using the ToolCollection object, with the slug of the collection you want to use. Then pass them as a list to initialize your agent, and start using them!


To speed up the start, tools are loaded only if called by the agent.